In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import re

# FINALE KOMPLETTE QUEST-SCRAPER VERSION
def fetch_quest_complete_with_js_coords(quest_id: int):
    """Komplette Quest-Extraktion mit automatischer JavaScript-Koordinaten-Extraktion"""
    chrome_options = Options()
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome(options=chrome_options)
    
    try:
        # 1. Quest-Seite besuchen
        url = f"https://database.turtle-wow.org/?quest={quest_id}"
        driver.get(url)
        time.sleep(2)
        
        # 2. Quest-Titel extrahieren
        title = None
        try:
            title_elements = driver.find_elements(By.CSS_SELECTOR, "h1")
            for elem in title_elements:
                text = elem.text.strip()
                if text and len(text) > 3:
                    title = text
                    break
        except:
            pass
        
        # 3. NPCs und deren URLs extrahieren
        npc_data = []
        try:
            npc_links = driver.find_elements(By.CSS_SELECTOR, "a[href*='npc=']")
            
            for link in npc_links:
                npc_name = link.text.strip()
                npc_url = link.get_attribute('href')
                
                # Extrahiere NPC-ID
                npc_id_match = re.search(r'npc=(\d+)', npc_url)
                if npc_id_match:
                    npc_id = int(npc_id_match.group(1))
                    npc_data.append({
                        "id": npc_id,
                        "name": npc_name,
                        "url": npc_url,
                        "coordinates": {"x": 0, "y": 0, "zone": "Unknown"}
                    })
                    
        except Exception as e:
            print(f"Fehler bei NPC-Extraktion: {e}")
        
        # 4. Objectives extrahieren (vereinfacht für diese Demo)
        objectives = []
        try:
            page_text = driver.find_element(By.TAG_NAME, "body").text
            lines = page_text.split('\n')
            
            description_text = ""
            in_description = False
            
            for line in lines:
                line = line.strip()
                if 'Description' in line:
                    in_description = True
                    continue
                elif 'Completion' in line or 'Gains' in line:
                    break
                elif in_description and line:
                    description_text += line + " "
            
            if description_text:
                desc = description_text.strip()
                if len(desc) > 20:
                    if len(desc) > 100:
                        desc = desc[:100] + "..."
                    objectives.append(desc)
                    
        except Exception as e:
            print(f"Fehler bei Objectives: {e}")
        
        quest_data = {
            "id": quest_id,
            "title": title or f"Quest {quest_id}",
            "npcs": npc_data,
            "objectives": objectives or ["Complete the quest"]
        }
        
        return quest_data
        
    finally:
        driver.quit()

def extract_js_coordinates_fast(npc_url: str):
    """Schnelle JavaScript-Koordinaten-Extraktion"""
    chrome_options = Options()
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome(options=chrome_options)
    
    try:
        driver.get(npc_url)
        time.sleep(2)
        
        # Finde JavaScript-Links mit onClick-Koordinaten
        js_links = driver.find_elements(By.CSS_SELECTOR, "a[href='javascript:;']")
        
        for link in js_links:
            onclick = link.get_attribute('onclick')
            if onclick and 'myMapper.update' in onclick:
                coord_match = re.search(r'coords:\s*\[\[(\d+\.?\d*),(\d+\.?\d*)', onclick)
                if coord_match:
                    x = float(coord_match.group(1))
                    y = float(coord_match.group(2))
                    return {"x": x, "y": y, "zone": "Azeroth"}
        
        return {"x": 0, "y": 0, "zone": "Unknown"}
        
    except Exception as e:
        print(f"Fehler bei Koordinaten-Extraktion: {e}")
        return {"x": 0, "y": 0, "zone": "Unknown"}
    finally:
        driver.quit()

def generate_complete_lua_quest(quest_id: int):
    """Komplette Quest-zu-Lua Konvertierung mit echten Koordinaten"""
    
    # 1. Quest-Daten extrahieren
    quest_data = fetch_quest_complete_with_js_coords(quest_id)
    
    # 2. Koordinaten für alle NPCs extrahieren
    for npc in quest_data['npcs']:
        coords = extract_js_coordinates_fast(npc['url'])
        npc['coordinates'] = coords
    
    # 3. Lua-Code generieren
    steps = []
    quest_id = quest_data["id"]
    title = quest_data["title"]
    npcs = quest_data["npcs"]
    
    if not npcs:
        return "# Keine NPC-Daten verfügbar"
    
    start_npc = npcs[0] if len(npcs) > 0 else None
    turnin_npc = npcs[1] if len(npcs) > 1 else start_npc
    
    # Accept Step
    if start_npc:
        coords = start_npc["coordinates"]
        steps.append(f'{{ action="accept", quest={quest_id}, name="{title}", npc="{start_npc["name"]}", coords={{x={coords["x"]},y={coords["y"]}}} }},')
    
    # Objective Steps
    for obj in quest_data["objectives"]:
        note = obj[:50] + "..." if len(obj) > 50 else obj
        steps.append(f'{{ action="info", quest={quest_id}, name="{title}", note="{note}", coords={{x=0,y=0}} }},')
    
    # Turn-in Step
    if turnin_npc:
        coords = turnin_npc["coordinates"]
        steps.append(f'{{ action="turnin", quest={quest_id}, name="{title}", npc="{turnin_npc["name"]}", coords={{x={coords["x"]},y={coords["y"]}}} }},')
    
    lua_code = "\n".join(steps)
    
    # Nur die Lua-Befehle ausgeben
    print(lua_code)
    
    return lua_code

# Test der kompletten finalen Version
final_lua = generate_complete_lua_quest(321)

{ action="accept", quest=321, name="Lightforge Iron", npc="Glorin Steelbrow", coords={x=10.58,y=60.59} },
{ action="info", quest=321, name="Lightforge Iron", note="The shipwrecks off the shore here are the doomed T...", coords={x=0,y=0} },
{ action="turnin", quest=321, name="Lightforge Iron", npc="Glorin Steelbrow", coords={x=10.58,y=60.59} },
